In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import cv2


input_dim = 5  # one-hot size
hidden_size = 4  # output from the LSTM. 5 to directly predict one-hot
batch_size = 50   # one sentence
sequence_length = 5  # |ihello| == 6
learning_rate = 0.0001
data_mean = 17.193384968996618
data_std = 63.52561803502815

In [2]:
X = tf.placeholder(tf.float32, [None, 200, 400, 1])
Y = tf.placeholder(tf.float32, [None, 4])

with tf.variable_scope("cnn"):
    
    # conv : 5, 200, 400, 16
    conv = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool : 5, 50, 99, 16
    pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 4], padding='SAME', strides=1)
    # conv2 
    conv2 = tf.layers.conv2d(inputs=pool, filters=64, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool2 
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4, 8], padding='SAME', strides=2)
    # conv3
    conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool3 
    pool3 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    
    flat = tf.reshape(pool3, [-1, 100*200*16])
    w = tf.get_variable("w", [hidden_size, hidden_size])
    b = tf.get_variable("b", [hidden_size])
    
    logit = tf.layers.dense(inputs=flat, units=4)
    outputs = tf.sigmoid(tf.matmul(logit, w) + b)
    
    loss = -tf.reduce_mean(Y*tf.log(outputs)+(1-Y)*(tf.log(1-outputs)))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [3]:
saver = tf.train.Saver()

with tf.Session() as sess:
    res = raw_input("Do you want addtional learning? (Name or Enter)")
    if res:
        saver.restore(sess, "./model/"+res+".ckpt")
    else:
        sess.run(tf.global_variables_initializer())
    model_name = raw_input("Model_Name: ")
    
    try:
        for epoch in range(10):
            for ims in range(1, 9):

                data_df = pd.read_csv('/mnt/d/imgs/img'+str(1)+'/log.csv', names=['name','key_out'])
                names = data_df['name'].values
                # keyouts = [8, 1, 4, 2, 15, ...]
                key_out = [eval(x) for x in data_df['key_out'].values]

                while len(names):
                    imgs = []
                    offset = min(batch_size, len(names))
                    batch_x = names[0:offset]
                    batch_y = key_out[0:offset]
                    names = names[offset:]
                    key_out = key_out[offset:]

                    for name in batch_x:
                        img = Image.open('/mnt/d/imgs/img'+str(1)+'/img/'+name)
                        data = np.array( img, dtype='uint8')
                        data = data.reshape([200,400,1])
                        data = (data-data_mean) / data_std
                        imgs.append(data)

                    sess.run(train, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
                    print sess.run([w, b ,loss], feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
            if epoch == 0:
                save_path = saver.save(sess, "./model/"+model_name+"_1.ckpt")
            if epoch == 4:
                save_path = saver.save(sess, "./model/"+model_name+"_5.ckpt")
            print 'epoch', epoch+1, sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
    finally:
        save_path = saver.save(sess, "./model/"+model_name+"_10.ckpt")

Do you want addtional learning? (Name or Enter)
Model_Name: test
[array([[-0.65848416,  0.8267823 , -0.27188328, -0.17639597],
       [-0.41834682, -0.6115481 , -0.34429815,  0.69140416],
       [-0.25388446,  0.34963307, -0.7190683 ,  0.7847299 ],
       [ 0.40914384,  0.6808732 , -0.32579288,  0.38398305]],
      dtype=float32), array([ 0.49389264,  0.50294584, -0.45566344, -0.40778834], dtype=float32), 1.0177189]
[array([[-0.658403  ,  0.8268821 , -0.27195555, -0.17630118],
       [-0.4182704 , -0.61146355, -0.34437162,  0.6914769 ],
       [-0.25397038,  0.3495371 , -0.7189977 ,  0.78462976],
       [ 0.4090725 ,  0.68082845, -0.32571742,  0.38392088]],
      dtype=float32), array([ 0.49386948,  0.5028843 , -0.45576185, -0.40784422], dtype=float32), 2.2023437]
[array([[-0.6583144 ,  0.8269823 , -0.2720008 , -0.17635117],
       [-0.4181903 , -0.61137706, -0.34440356,  0.69141746],
       [-0.25406212,  0.3494444 , -0.71895057,  0.7846723 ],
       [ 0.4089892 ,  0.6807581 , -0.3256

[array([[-0.6573687 ,  0.8266803 , -0.27196413, -0.17627887],
       [-0.41749313, -0.61112905, -0.34533185,  0.69035685],
       [-0.2545019 ,  0.34946316, -0.7174377 ,  0.78487045],
       [ 0.40835243,  0.6807369 , -0.324336  ,  0.38445747]],
      dtype=float32), array([ 0.49430284,  0.5020258 , -0.45727223, -0.4090713 ], dtype=float32), 2.989321]
[array([[-0.6573518 ,  0.82665193,         nan, -0.17626554],
       [-0.41746753, -0.6111171 ,         nan,  0.69030905],
       [-0.25448897,  0.3494995 ,         nan,  0.7848297 ],
       [ 0.40835735,  0.6807943 ,         nan,  0.3844484 ]],
      dtype=float32), array([ 0.49431488,  0.50197905,         nan, -0.40911117], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan

[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan

[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), array([nan, nan, nan, nan], dtype=float32), nan]
[array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan

KeyboardInterrupt: 